In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-2.3.3/spark-2.3.3-bin-hadoop2.7.tgz
!tar xf spark-2.3.3-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.3-bin-hadoop2.7"

In [0]:
import os
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [0]:
# Reading dataframe from json file
df = spark.read.json('people.json')

In [0]:
# View the entries in dataframe
df.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
# Print the schema of dataframe
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [0]:
# To see the column names
df.columns

['age', 'name']

In [0]:
# View the statistical summary of numeric columns
df.describe().show()

+-------+------------------+-------+
|summary|               age|   name|
+-------+------------------+-------+
|  count|                 2|      3|
|   mean|              24.5|   null|
| stddev|7.7781745930520225|   null|
|    min|                19|   Andy|
|    max|                30|Michael|
+-------+------------------+-------+



In [0]:
# Defining our own schema
from pyspark.sql.types import (StructField,StringType,
                               IntegerType,StructType)

#(field_name,type,nullable)
data_schema = [StructField('age',IntegerType(),True),
               StructField('name',StringType(),True)]

final_struc = StructType(fields=data_schema)

In [0]:
df = spark.read.json('people.json',schema=final_struc)
df.printSchema()

In [0]:
# to select dataframe with single column
# using select method
df.select('age').show()

+----+
| age|
+----+
|null|
|  30|
|  19|
+----+



In [0]:
# to select multiple columns
df.select(['age','name']).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [0]:
# to get the first two row objects in dataframe
df.head(2)

[Row(age=None, name='Michael'), Row(age=30, name='Andy')]

In [0]:
# to make a new column with existing col
df.withColumn('double_age',df['age']*2).show()

+----+-------+----------+
| age|   name|double_age|
+----+-------+----------+
|null|Michael|      null|
|  30|   Andy|        60|
|  19| Justin|        38|
+----+-------+----------+



In [0]:
# to rename a column in dataframe
df.withColumnRenamed('age','my_new_age').show()

+----------+-------+
|my_new_age|   name|
+----------+-------+
|      null|Michael|
|        30|   Andy|
|        19| Justin|
+----------+-------+



In [0]:
# performing SQL queries on dataframe
# register the dataframe as sql temporary view
df.createOrReplaceTempView('people')
results = spark.sql('SELECT * FROM people WHERE age=30')
results.show()

+---+----+
|age|name|
+---+----+
| 30|Andy|
+---+----+

